## imports

In [1]:
import os, time
from glob import glob
import numpy as np

from model.VariationalAutoencoder import VariationalAutoencoder
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


# VAE

## model variables

In [2]:
DATETIME = time.strftime("%Y%m%d-%H%M%S")

LEARNING_RATE = 0.0005
R_LOSS_FACTOR = 1000

cloud_training = False

if cloud_training :
    BATCH_SIZE = 200
    EPOCHS = 50
    DATA_FOLDER = '/data'
    RUN_FOLDER = '/tmp/run/'
else:
    BATCH_SIZE = 5
    EPOCHS = 5
    DATA_FOLDER = '/Volumes/My Passport/PFM/fastcheck'
    RUN_FOLDER =  '/Volumes/My Passport/PFM/run/'
    
run_id = '0001'
data_name = 'cells'
RUN_FOLDER += '_'.join([run_id, data_name, DATETIME])

INPUT_DIM = (128,128,3)

## training setup

In [3]:
t = VariationalAutoencoder()

t.compile(learning_rate=LEARNING_RATE, r_loss_factor=R_LOSS_FACTOR)

t.encoder.summary()
t.decoder.summary()

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))
    

filenames = np.array(glob(os.path.join(DATA_FOLDER, '*/*.png')))

NUM_IMAGES = len(filenames)

print('Total images: ' + str(NUM_IMAGES))

steps_per_epoch = NUM_IMAGES / BATCH_SIZE
data_gen = ImageDataGenerator(rescale=1./255) # 8 BITS X CHANNEL
data_flow = data_gen.flow_from_directory(DATA_FOLDER
                                         , target_size = INPUT_DIM[:2]
                                         , batch_size = BATCH_SIZE
                                         , shuffle = True
                                         , class_mode = 'input'
                                         , subset = "training")

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
encoder_conv_1 (Conv2D)         (None, 64, 64, 32)   896         encoder_input[0][0]              
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 64, 64, 32)   0           encoder_conv_1[0][0]             
__________________________________________________________________________________________________
encoder_conv_2 (Conv2D)         (None, 32, 32, 64)   18496       leaky_re_lu_1[0][0]              
____________________________________________________________________________________________

## train

In [4]:
t.train_with_generator(data_flow = data_flow, epochs = EPOCHS, steps_per_epoch = NUM_IMAGES / BATCH_SIZE, run_folder = RUN_FOLDER, print_every_n_batches = 10)

Epoch 1/5
5/4 [===================================] - 19s 4s/step - loss: 8.7382 - vae_r_loss: 7.7696 - vae_kl_loss: 0.0237

Epoch 00001: saving model to /Volumes/My Passport/PFM/run/0001_cells_20200408-140656/weights/weights-001-7.79.h5

Epoch 00001: saving model to /Volumes/My Passport/PFM/run/0001_cells_20200408-140656/weights/weights.h5
Epoch 2/5
5/4 [===================================] - 15s 3s/step - loss: 5.1332 - vae_r_loss: 4.9989 - vae_kl_loss: 0.0283

Epoch 00002: saving model to /Volumes/My Passport/PFM/run/0001_cells_20200408-140656/weights/weights-002-5.03.h5

Epoch 00002: saving model to /Volumes/My Passport/PFM/run/0001_cells_20200408-140656/weights/weights.h5
Epoch 3/5
5/4 [===================================] - 14s 3s/step - loss: 3.9737 - vae_r_loss: 3.9840 - vae_kl_loss: 0.0251

Epoch 00003: saving model to /Volumes/My Passport/PFM/run/0001_cells_20200408-140656/weights/weights-003-4.01.h5

Epoch 00003: saving model to /Volumes/My Passport/PFM/run/0001_cells_202004